# Prediction with LSTM model (v.02)
<p style='text-align: right;'>with selectd3.csv</p>

* history
  * 2025/05/16 PM06:56 : 4번째 모델의 8번째 데이터를 사용

In [ ]:
!pip install scikit-learn
!pip install tensorflow
from datetime import datetime
print(f'restart kernel... {datetime.now()}')

In [1]:
def restart_kernel():
    # Restart the kernet after libraries are loaded.
    import IPython
    from datetime import datetime
    print(f'restart kernel... {datetime.now()}')
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

restart_kernel()

restart kernel... 2025-05-21 09:38:59.452489


In [1]:
# load dependacies
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Input

from globalvar import *
from data import DataLoader
from model1 import PredictLSTM1
from model2 import PredictLSTM2
from model3 import PredictLSTM3
from model4 import PredictLSTM4

import argparse
import matplotlib.pyplot as plt

from util_pred import print_data
from util_pred import flat_data_with_sum, flat_data, get_frequency
from util_pred import save_model, import_mode
from util_pred import get_random_in_list
from util_pred import print_data_with_sort
from util_pred import get_sorted_n_values
from util_pred import dict_key_count
from util_pred import print_list
from util_pred import print_dict_list
from util_pred import print_title
from activation import ActivationOutput, RecurrentActivation
from datetime import datetime

from operate import create_model, create_model_v1, get_predicted, create_model_v2

print(f"imported library. ({datetime.now()})")

2025-05-21 09:39:04.855653: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


imported library. (2025-05-21 09:39:06.942249)


In [3]:
##
## 모델링 환경 설정
##
window=1 # default = 1 , help = "time stamps"
data_dir='./selectd3.csv'
    
mode='predict' # help = "back-test or predict")
mode2='sampling' # help = "greed or sampling")
verb='verbose' # help = "verbose or not_verb")
    
trial=20 # help = "how much trials to generate")
training_length=0.95 # default = 0.9)
epoch=20 # default = 3
batch=1 # default = 1
model_type='lstm4' # help = "lstm1 or lstm2")
hid_dim = 50
from_pos = 0
last = [[7, 9, 24, 40, 42, 44],
        [3, 6, 7, 11, 12, 17],
        [3, 13, 28, 34, 38, 42],
        [5, 12, 24, 29, 32, 42]]
print(f"finished to set environemnt. ({datetime.now()})")
MAX_MODEL_CNT = 5

finished to set environemnt. (2025-05-21 09:39:17.684928)


In [4]:
# data prepare and set base model
dataset = DataLoader(data_dir=data_dir,
                     training_length=training_length,
                     window_prev=window,
                     mode=mode,
                     from_pos=from_pos
                    )

dataset_dicts = {
    1: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=0
                  ),
    2: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=300
                  )
    }
print(f'test_X = {dataset_dicts[1].test_X}')
print(f'completed to load data. {datetime.now()}')

DataLoader.preproc_csv origin = 571, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [1169    5   12 ...   26   39   42]
 [1170    3   13 ...   34   38   42]
 [1171    3    6 ...   11   12   17]]
rDataLoader.preproc_csv aw_np_proc = 571, [[16 19 31 34 35]
 [14 22 27 30 38]
 [19 25 26 27 43]
 ...
 [12 24 26 39 42]
 [13 28 34 38 42]
 [ 6  7 11 12 17]]
DataLoader.preproc_csv origin = 571, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [1169    5   12 ...   26   39   42]
 [1170    3   13 ...   34   38   42]
 [1171    3    6 ...   11   12   17]]
rDataLoader.preproc_csv aw_np_proc = 571, [[16 19 31 34 35]
 [14 22 27 30 38]
 [19 25 26 27 43]
 ...
 [12 24 26 39 42]
 [13 28 34 38 42]
 [ 6  7 11 12 17]]
DataLoader.preproc_csv origin = 571, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [116

In [5]:
###########
## layers
## LSTM Neural 계층 선언
###########
"""
Activation (Output):
    linear: No activation, output is directly passed through.
    relu: Rectified Linear Unit, max(x, 0).
    sigmoid: Sigmoid function, output between 0 and 1.
    tanh: Hyperbolic tangent, output between -1 and 1.
    softmax: Normalizes output to a probability distribution.
    elu: Exponential Linear Unit.
    selu: Scaled Exponential Linear Unit.

Recurrent Activation:
    sigmoid: Commonly used for gates in LSTM.
    hard_sigmoid: A faster, less computationally expensive version of sigmoid.
    tanh: Can be used, but sigmoid is more typical for gates.
"""
datasets = [dataset_dicts[1], # 0
            dataset_dicts[2], # 1
           ]
# lstm_units = [[45, 45, 45, 45, 45, 45],
#               [45, 45, 45]]
lstm_units=[[(45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.sigmoid.name),
             (45, ActivationOutput.sigmoid.name),
             (45, ActivationOutput.elu.name)
             ],
            [(45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name)
             ],
            [(45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.sigmoid.name),
             (45, ActivationOutput.sigmoid.name),
             (45, ActivationOutput.elu.name)
             ],
            [(45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name)
             ],
            [(45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name)
             ]]
dense_units = [[(45, ActivationOutput.selu.name),
                (45, ActivationOutput.elu.name),
                (45, ActivationOutput.elu.name),],
               [(45, ActivationOutput.selu.name),],
               [(45, ActivationOutput.selu.name),
                (45, ActivationOutput.elu.name),
                (45, ActivationOutput.elu.name),],
               [(45, ActivationOutput.tanh.name),
               ],
               []
              ]
steps = [45, 45,]
metrics = [["accuracy"]]
dropout = [0, 0]
learning_rate = [0.01, 0.05]
last_lstm_return_sequences = [False]
loss = ["binary_crossentropy", "mse"]
output_dense_activation = [ActivationOutput.elu.name, #0
                           ActivationOutput.selu.name, #1
                           ActivationOutput.sigmoid.name, #2
                           ActivationOutput.tanh.name, #3
                           ActivationOutput.softmax.name, #4
                           ActivationOutput.relu.name, #5
                           ActivationOutput.linear.name #6
                          ]
epochs = [10, #0
          20, #1
          25, #2
          50, #3
          100] #4

rand_seed=[0, datetime.now().timestamp(), 0]


lstm_args = {1: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[0],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[0],
                          "dense_units": dense_units[0],
                          "output_dense_activation": output_dense_activation[0],
                          "loss": loss[0],
                          "rand_seed": rand_seed[0]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 1
                           },
                 "dataset": datasets[0]
                 },
             2: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[0],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[1],
                          "dense_units": dense_units[1],
                          "output_dense_activation": output_dense_activation[0],
                          "loss": loss[0],
                          "rand_seed": rand_seed[1]
                          },
                 "train": {"epochs": epochs[1],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 1
                           },
                 "dataset": datasets[0]
                 },
             3: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[0],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[1],
                          "dense_units": dense_units[1],
                          "output_dense_activation": output_dense_activation[0],
                          "loss": loss[0],
                          "rand_seed": rand_seed[1]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 1
                           },
                 "dataset": datasets[0]
                 },
             4: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[1],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[2],
                          "dense_units": dense_units[2],
                          "output_dense_activation": output_dense_activation[3],
                          "loss": loss[1],
                          "rand_seed": rand_seed[2]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 3,
                           "steps_per_epoch": 100
                           },
                 "dataset": datasets[0]
                 },
             5: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[1],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[3],
                          "dense_units": dense_units[3],
                          "output_dense_activation": output_dense_activation[3],
                          "loss": loss[1],
                          "rand_seed": rand_seed[2]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 3,
                           "steps_per_epoch": 100
                           },
                 "dataset": datasets[0]
                 },
             }


models = [None for i in range(MAX_MODEL_CNT)]

matched_cnts = {}
selected_fives = {}
matched_list = {}
predicted_all = {}

print(f'completed to set evn for all models. {datetime.now()}')

completed to set evn for all models. 2025-05-21 09:39:19.717742


In [6]:
single_run = False
if single_run:
    model = create_model_v2(id=1,
                            model_type="lstm4",
                            lstm_info=lstm_args[1],
                            dataset=datasets[1],
                            verbose=True)

In [7]:
# all : create model and training
print(f'start to train all models. {datetime.now()}')
for i in range(MAX_MODEL_CNT):
    print(f'{i+1}\'s training. status=start {datetime.now()}')
    model = create_model_v2(id=i,
                         model_type="lstm4",
                         lstm_info=lstm_args[i+1],
                         dataset=lstm_args[i+1]["dataset"],
                         verbose=True
                        )
    models[i] = (model, lstm_args[i+1])
    print(f'{i+1}\'s training. status=end {datetime.now()}')
print(f'completed to train all models. {datetime.now()}')

start to train all models. 2025-05-21 09:39:28.075230
1's training. status=start 2025-05-21 09:39:28.075412
create_model_v2.model1 start to create model. 2025-05-21 09:39:28.075439
PredictLSTM4.create_model [{'learning_rate': 0.01, 'dropout': 0, 'steps': 45, 'metrics': ['accuracy'], 'last_lstm_return_sequences': False, 'lstm_units': [(45, 'selu'), (45, 'selu'), (45, 'sigmoid'), (45, 'sigmoid'), (45, 'elu')], 'dense_units': [(45, 'selu'), (45, 'elu'), (45, 'elu')], 'output_dense_activation': 'elu', 'loss': 'binary_crossentropy', 'rand_seed': 0}]
lstm unit : 1, (45, 'selu')
lstm unit : 2, (45, 'selu')
lstm unit : 3, (45, 'sigmoid')
lstm unit : 4, (45, 'sigmoid')
lstm unit : 5, (45, 'elu')
loss = binary_crossentropy
create_model_v2.model1 comleted to generate model. 2025-05-21 09:39:28.199524
create_model_v2.model1 start to train model. 2025-05-21 09:39:28.199563
create_model_v2.model1   epochs = 50 2025-05-21 09:39:28.199580
verbose=2
model=<Sequential name=sequential, built=True>
num_ep

In [66]:
print(f'check to create model. {datetime.now()}')

check to create model. 2025-05-21 22:15:58.435631


In [9]:
# all : predict #1
greed_prediction = [None for i in range(MAX_MODEL_CNT)]
random_pred = [None for i in range(MAX_MODEL_CNT)]
for i in range(MAX_MODEL_CNT):
    greed_prediction[i] = models[i][0].predict_numbers("greed", trial=1)
    random_pred[i] = models[i][0].predict_randomely(trial=1)
    print(f'{i+1}\'s greed_prediction = {greed_prediction[i]}')
    print(f'{i+1}\'s random_pred = {random_pred[i]}')
print(f'comleted to predict model. {datetime.now()}')

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step
1's greed_prediction = [array([45, 41, 39,  5, 30,  1])]
1's random_pred = [[2, 8, 23, 37, 30, 26]]
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2's greed_prediction = [array([27, 14, 38,  5, 30, 34])]
2's random_pred = [[22, 12, 21, 3, 28, 41]]
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
3's greed_prediction = [array([ 8, 38, 22, 14,  6, 33])]
3's random_pred = [[10, 23, 16, 30, 42, 37]]
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step
4's greed_prediction = [array([27, 34, 21, 41, 23, 37])]
4's random_pred = [[33, 17, 4, 3, 25, 43]]
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step
5's greed_prediction = [array([41, 21, 33, 45, 19, 37])]
5's random_pred = [[8, 29, 34, 38, 42, 35]]
comleted to predict model. 2025-05-21 09:43:10.093489


In [10]:
single_run = False
if single_run:
    matched_cnts, selected_fives, matched_list, predicted_all = repeat_predict_all_model(trial=20)

In [13]:
print(matched_cnts)
print(matched_list)
for i in range(MAX_MODEL_CNT):
    print(models[i][0].model.summary())

{}
{}


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 1, 45)          │        16,380 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 1, 45)          │        16,380 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 1, 45)          │        16,380 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 1, 45)          │        16,380 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 45)             │        16,380 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 45)             │         2,070 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 45)             │         2,070 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 45)             │         2,070 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 45)             │         2,070 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 270,542 (1.03 MB)

 Trainable params: 90,180 (352.27 KB)

...

 Total params: 356,402 (1.36 MB)

 Trainable params: 118,800 (464.06 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 237,602 (928.14 KB)

None


In [11]:
import json
import os
from datetime import datetime

def listinlist_2_strinlist(list_in_list):
    """ listinlist_2_strinlist """
    str_in_list = []
    for l in list_in_list:
        str_in_list.append(",".join(l))
    return str_in_list      


def change_matched_info(matched_count, matched_list):
    counts = {}
    cnt = 1
    for c in matched_count:
        counts[cnt] = ",".join([ str(i) for i in c])
        cnt += 1
    cnt = 1
    datas = {}
    for d in matched_list:
        datas[cnt] = d
        cnt += 1
    return counts, datas

def write_json(trial, matched_count, matched_list, append=False):
    suffix = datetime.now().strftime('%y%m%d_%H')
    file_name = f"matched_{suffix}.json"
    matched_dict = {}
    if append:
        if os.path.isfile(file_name):
            with open(file_name, "r") as file:
                matched_dict = json.load(file)
                file.close()
    counts, datas = change_matched_info(matched_count, matched_list)
    if len(counts) > 0 and len(datas) > 0:
        matched_dict[trial] = {}
        matched_dict[trial]["counts"] =  counts
        matched_dict[trial]["datas"] = datas
        print(f'matched_dict = {matched_dict}')
        with open(file_name, "w") as file:
            json.dump(matched_dict, file, indent=4)

In [12]:
# all with trial : print predicted
run = False
if run:
    write_json(trial = 3, matched_count=matched_cnts, matched_list=matched_list, append=True)
    print_dict_list(title="matched_cnt", datas=matched_cnts)
    for i in range(MAX_MODEL_CNT):
        print_list(title=f"matched_list[{i+1}]", datas=matched_list[i])

| 회차 | 3이상 ||| 0이 없음 |
| -- | -- | -- | -- | -- |
|  1 | N ||| 2 |
|  2 | N ||| 1 |
|  3 | **1**, 2 ||| **1** |
|  4 | N ||| N |
|  5 | N ||| N |
|  6 | N ||| N |
|  7 | N ||| N |
|  8 | 2 ||| N |

|  8 | 1 ||| N |
|  8 | N ||| N |
|  8 | N ||| N |
|  8 | 2 ||| N |




{14:6, 13:8, 12:4, _11:10_, 10:3, 9:1, 8:6, 7:5, 6:4, 5:9, 4:4, 3:7, 2:7, 1:4}
## 4시 40분경 데이터
- {2: 6, 5: 5, 6: 4, 4: 3, 3: 3, 1: 2}
- {6: 4, 5: 5, 4: 3, 3: 3, 2: 6, 1: 2}

## 5시 20분경 데이터
- {4: 5, 5: 5, 1: 4, 6: 3, 3: 2}
- {6: 3, 5: 5, 4: 5, 3: 2, 1: 4}

In [13]:
# all (SMALL): predict #2
def predict_small(trial, last, models):
    matched_cnts_small = [0 for i in range(MAX_MODEL_CNT)]
    selected_fives_small = [None for i in range(MAX_MODEL_CNT)]
    matched_list_small = [None for i in range(MAX_MODEL_CNT)]
    predicted_all_small = [None for i in range(MAX_MODEL_CNT)]

    for i in range(MAX_MODEL_CNT):
        title = f"history #{i+1}"
        model_num = i
        if models[model_num] is None:
            print(f"model #{model_num+1} is None")
        else:
            models[model_num][0].verb = "None"
            matched_cnts_small[model_num], \
            selected_fives_small[model_num], \
            matched_list_small[model_num], \
            predicted_all_small[model_num] = get_predicted(
                title=title,
                model=models[model_num][0],
                mode=mode2,
                use_pre=False,
                last=last,
                verbose=False,
                trial=5
            )
    print(f'predicted time : trial={trial} date={datetime.now()}')
    return matched_cnts_small, matched_list_small

In [31]:
# all (SMALL): print predicted
def print_predict_small(matched_cnts_small, matched_list_small):
    print(f'print time : {datetime.now()}')
    #write_json(trial = 3, matched_count=matched_cnts_small, matched_list=matched_list_small, append=True)
    print_dict_list(title="matched_cnt", datas=matched_cnts_small)
    for i in range(MAX_MODEL_CNT):
        print_list(title=f"matched_list[{i+1}]", datas=matched_list_small[i])

In [67]:
trial = 1
matched_cnts_all = []
matched_list = []
for t in range(1, trial+1):
    matched_cnts_small, matched_list_small = predict_small(trial=t, 
                                                           last=last,
                                                           models=models)
    #print_predict_small(matched_cnts_small, matched_list_small)
    matched_cnts_all.append(matched_cnts_small)
    matched_list.append(matched_list_small)

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
predicted time : trial=1 date=2025-05-21 22:16:34.527810


In [68]:
# 2 2 2 2 2 -> 2 1 1 2 2
cnt = 1
for matched in matched_list:
    print(f'print predicted data [{cnt}]. {datetime.now()}')
    cnt1 = 1
    for m in matched:
        print_list(title=f"total_matched_{cnt}_model_{cnt1}", datas=m)
        cnt1 += 1
    cnt += 1

print predicted data [1]. 2025-05-21 22:16:43.576317
total_matched_1_model_1
	([7, 10, 17, 18, 25, 41], [7])
	([9, 21, 6, 37, 17, 41], [9])
	([16, 20, 22, 23, 32, 41], [])
	([8, 17, 22, 39, 42, 43], [42])
	([44, 8, 17, 10, 37, 24], [24, 44])
total_matched_1_model_2
	([11, 21, 28, 15, 38, 41], [])
	([8, 13, 20, 32, 36, 42], [42])
	([2, 11, 13, 20, 24, 36], [24])
	([10, 43, 17, 7, 41, 1], [7])
	([2, 3, 20, 36, 30, 25], [])
total_matched_1_model_3
	([4, 22, 24, 25, 39, 41], [24])
	([6, 15, 19, 20, 24, 31], [24])
	([2, 8, 14, 23, 31, 40], [40])
	([42, 28, 14, 23, 10, 1], [42])
	([24, 27, 20, 29, 35, 15], [24])
total_matched_1_model_4
	([37, 8, 28, 39, 38, 44], [44])
	([39, 5, 19, 43, 31, 45], [])
	([2, 3, 9, 32, 33, 36], [9])
	([2, 3, 7, 32, 37, 45], [7])
	([1, 2, 7, 18, 32, 42], [7, 42])
total_matched_1_model_5
	([39, 24, 10, 29, 8, 44], [24, 44])
	([5, 20, 25, 28, 33, 35], [])
	([8, 12, 23, 24, 29, 34], [24])
	([1, 13, 24, 31, 32, 45], [24])
	([3, 11, 19, 20, 27, 40], [40])


In [65]:
# 2 2 2 2 2 -> 2 1 1 2 2
cnt = 1
for matched in matched_list:
    print(f'print predicted data [{cnt}]. {datetime.now()}')
    cnt1 = 1
    for m in matched:
        print_list(title=f"total_matched_{cnt}_model_{cnt1}", datas=m)
        cnt1 += 1
    cnt += 1

print predicted data [1]. 2025-05-21 10:00:13.894863
total_matched_1_model_1
	([5, 10, 13, 17, 18, 40], [40])
	([39, 20, 12, 9, 17, 6], [9])
	([10, 11, 13, 21, 28, 41], [])
	([44, 41, 10, 6, 2, 15], [44])
	([6, 19, 21, 24, 25, 45], [24])
total_matched_1_model_2
	([44, 24, 2, 38, 17, 26], [24, 44])
	([45, 17, 12, 11, 29, 23], [])
	([9, 18, 26, 27, 29, 37], [9])
	([20, 29, 30, 33, 38, 40], [40])
	([6, 7, 12, 21, 24, 45], [7, 24])
total_matched_1_model_3
	([4, 8, 14, 18, 22, 24], [24])
	([12, 9, 4, 36, 17, 22], [9])
	([10, 12, 17, 18, 35, 40], [40])
	([9, 10, 15, 25, 27, 40], [9, 40])
	([24, 5, 29, 2, 17, 39], [24])
total_matched_1_model_4
	([26, 30, 32, 35, 41, 45], [])
	([2, 26, 12, 11, 29, 40], [40])
	([3, 4, 16, 19, 29, 35], [])
	([6, 9, 16, 17, 27, 38], [9])
	([11, 12, 14, 15, 24, 34], [24])
total_matched_1_model_5
	([14, 16, 25, 32, 34, 45], [])
	([14, 16, 17, 27, 30, 37], [])
	([7, 9, 12, 17, 23, 45], [7, 9])
	([14, 32, 34, 6, 23, 12], [])
	([43, 17, 25, 8, 23, 3], [])


In [62]:
# 2 2 2 2 2 -> 2 1 1 2 2
cnt = 1
for matched in matched_list:
    print(f'print predicted data [{cnt}]. {datetime.now()}')
    cnt1 = 1
    for m in matched:
        print_list(title=f"total_matched_{cnt}_model_{cnt1}", datas=m)
        cnt1 += 1
    cnt += 1

print predicted data [1]. 2025-05-21 09:59:48.739001
total_matched_1_model_1
	([22, 40, 9, 18, 3, 13], [9, 40])
	([9, 18, 21, 38, 40, 43], [9, 40])
	([1, 10, 20, 29, 36, 44], [44])
	([8, 9, 17, 31, 35, 41], [9])
	([3, 23, 25, 29, 30, 36], [])
total_matched_1_model_2
	([12, 1, 34, 18, 37, 20], [])
	([7, 15, 19, 24, 25, 37], [7, 24])
	([17, 9, 22, 31, 33, 41], [9])
	([30, 7, 44, 39, 26, 31], [7, 44])
	([13, 16, 25, 33, 37, 40], [40])
total_matched_1_model_3
	([1, 10, 21, 27, 30, 32], [])
	([1, 10, 18, 22, 34, 42], [42])
	([37, 41, 18, 39, 14, 3], [])
	([9, 21, 26, 29, 32, 34], [9])
	([28, 13, 30, 23, 19, 2], [])
total_matched_1_model_4
	([1, 16, 7, 40, 37, 44], [7, 40, 44])
	([4, 27, 34, 37, 42, 43], [42])
	([8, 17, 21, 28, 35, 44], [44])
	([5, 26, 29, 39, 43, 44], [44])
	([4, 7, 14, 26, 33, 40], [7, 40])
total_matched_1_model_5
	([30, 25, 36, 9, 32, 44], [9, 44])
	([16, 26, 29, 30, 37, 43], [])
	([7, 14, 16, 30, 43, 44], [7, 44])
	([7, 11, 13, 16, 30, 38], [7])
	([4, 14, 16, 17, 25, 40]

In [60]:
# 2 2 2 2 2 -> 2 2 2 2 2
cnt = 1
for matched in matched_list:
    print(f'print predicted data [{cnt}]. {datetime.now()}')
    cnt1 = 1
    for m in matched:
        print_list(title=f"total_matched_{cnt}_model_{cnt1}", datas=m)
        cnt1 += 1
    cnt += 1

print predicted data [1]. 2025-05-21 09:59:16.556950
total_matched_1_model_1
	([14, 23, 24, 26, 37, 41], [24])
	([1, 23, 30, 38, 41, 45], [])
	([8, 9, 22, 25, 27, 30], [9])
	([16, 39, 13, 14, 9, 34], [9])
	([10, 11, 20, 30, 38, 42], [42])
total_matched_1_model_2
	([24, 32, 37, 8, 16, 15], [24])
	([3, 11, 21, 31, 33, 36], [])
	([12, 23, 24, 25, 29, 45], [24])
	([8, 40, 18, 5, 36, 3], [40])
	([4, 7, 12, 22, 31, 41], [7])
total_matched_1_model_3
	([18, 44, 36, 25, 7, 30], [7, 44])
	([42, 23, 34, 6, 41, 35], [42])
	([1, 10, 16, 22, 28, 45], [])
	([20, 17, 36, 42, 6, 40], [40, 42])
	([8, 12, 16, 18, 19, 20], [])
total_matched_1_model_4
	([2, 12, 27, 35, 39, 41], [])
	([1, 45, 2, 21, 37, 23], [])
	([1, 5, 17, 26, 27, 37], [])
	([1, 8, 31, 37, 40, 45], [40])
	([2, 14, 22, 27, 31, 32], [])
total_matched_1_model_5
	([40, 33, 32, 15, 23, 28], [40])
	([2, 9, 19, 32, 34, 45], [9])
	([8, 29, 32, 35, 38, 44], [44])
	([2, 8, 9, 19, 36, 44], [9, 44])
	([9, 14, 16, 22, 24, 40], [9, 24, 40])


In [54]:
# 3 1 1 2 2
cnt = 1
for matched in matched_list:
    print(f'print predicted data [{cnt}]. {datetime.now()}')
    cnt1 = 1
    for m in matched:
        print_list(title=f"total_matched_{cnt}_model_{cnt1}", datas=m)
        cnt1 += 1
    cnt += 1

print predicted data [1]. 2025-05-21 09:53:53.575525
total_matched_1_model_1
	([25, 7, 5, 21, 42, 19], [7, 42])
	([9, 11, 22, 31, 36, 40], [9, 40])
	([38, 11, 39, 26, 9, 35], [9])
	([2, 5, 10, 21, 27, 45], [])
	([8, 9, 16, 28, 34, 43], [9])
total_matched_1_model_2
	([3, 16, 20, 26, 39, 42], [42])
	([3, 5, 8, 12, 23, 27], [])
	([11, 22, 23, 26, 42, 44], [42, 44])
	([14, 19, 32, 25, 3, 12], [])
	([9, 12, 15, 24, 37, 44], [9, 24, 44])
total_matched_1_model_3
	([4, 38, 22, 42, 34, 26], [42])
	([2, 4, 10, 17, 28, 33], [])
	([45, 11, 17, 10, 24, 38], [24])
	([2, 12, 22, 26, 30, 35], [])
	([9, 10, 17, 21, 28, 36], [9])
total_matched_1_model_4
	([23, 14, 15, 39, 43, 22], [])
	([12, 13, 16, 23, 29, 30], [])
	([14, 15, 18, 21, 23, 33], [])
	([4, 5, 22, 26, 27, 42], [42])
	([14, 31, 18, 1, 24, 41], [24])
total_matched_1_model_5
	([2, 4, 31, 36, 42, 43], [42])
	([37, 9, 27, 19, 21, 11], [9])
	([19, 25, 30, 31, 43, 45], [])
	([11, 17, 24, 38, 39, 41], [24])
	([8, 26, 29, 30, 40, 44], [40, 44])


In [48]:
# 2 2 1 2 3
cnt = 1
for matched in matched_list:
    print(f'print predicted data [{cnt}]. {datetime.now()}')
    cnt1 = 1
    for m in matched:
        print_list(title=f"total_matched_{cnt}_model_{cnt1}", datas=m)
        cnt1 += 1
    cnt += 1

print predicted data [1]. 2025-05-21 09:51:44.817140
total_matched_1_model_1
	([20, 42, 25, 12, 16, 35], [42])
	([5, 7, 12, 22, 30, 40], [7, 40])
	([6, 24, 28, 38, 45, 43], [24])
	([2, 13, 14, 18, 21, 44], [44])
	([4, 6, 20, 25, 43, 44], [44])
total_matched_1_model_2
	([5, 8, 11, 16, 24, 35], [24])
	([1, 2, 7, 15, 28, 32], [7])
	([31, 2, 7, 14, 9, 33], [7, 9])
	([6, 17, 29, 30, 34, 40], [40])
	([40, 2, 5, 39, 37, 18], [40])
total_matched_1_model_3
	([2, 14, 31, 32, 37, 40], [40])
	([3, 24, 27, 45, 12, 44], [24, 44])
	([9, 11, 16, 28, 33, 42], [9, 42])
	([3, 14, 18, 21, 27, 34], [])
	([17, 9, 42, 5, 45, 24], [9, 24, 42])
total_matched_1_model_4
	([4, 8, 16, 25, 26, 37], [])
	([1, 9, 17, 36, 43, 45], [9])
	([27, 9, 28, 7, 20, 34], [7, 9])
	([7, 12, 36, 28, 21, 24], [7, 24])
	([3, 6, 16, 22, 30, 34], [])
total_matched_1_model_5
	([9, 11, 14, 26, 35, 43], [9])
	([3, 4, 9, 25, 31, 34], [9])
	([6, 21, 22, 23, 36, 44], [44])
	([2, 5, 11, 13, 42, 45], [42])
	([1, 3, 14, 35, 40, 42], [40, 42])


In [47]:
# 3 2 2 2 2
cnt = 1
for matched in matched_list:
    print(f'print predicted data [{cnt}]. {datetime.now()}')
    cnt1 = 1
    for m in matched:
        print_list(title=f"total_matched_{cnt}_model_{cnt1}", datas=m)
        cnt1 += 1
    cnt += 1

print predicted data [1]. 2025-05-21 09:51:12.803096
total_matched_1_model_1
	([20, 42, 25, 12, 16, 35], [42])
	([5, 7, 12, 22, 30, 40], [7, 40])
	([6, 24, 28, 38, 45, 43], [24])
	([2, 13, 14, 18, 21, 44], [44])
	([4, 6, 20, 25, 43, 44], [44])
total_matched_1_model_2
	([5, 8, 11, 16, 24, 35], [24])
	([1, 2, 7, 15, 28, 32], [7])
	([31, 2, 7, 14, 9, 33], [7, 9])
	([6, 17, 29, 30, 34, 40], [40])
	([40, 2, 5, 39, 37, 18], [40])
total_matched_1_model_3
	([2, 14, 31, 32, 37, 40], [40])
	([3, 24, 27, 45, 12, 44], [24, 44])
	([9, 11, 16, 28, 33, 42], [9, 42])
	([3, 14, 18, 21, 27, 34], [])
	([17, 9, 42, 5, 45, 24], [9, 24, 42])
total_matched_1_model_4
	([4, 8, 16, 25, 26, 37], [])
	([1, 9, 17, 36, 43, 45], [9])
	([27, 9, 28, 7, 20, 34], [7, 9])
	([7, 12, 36, 28, 21, 24], [7, 24])
	([3, 6, 16, 22, 30, 34], [])
total_matched_1_model_5
	([9, 11, 14, 26, 35, 43], [9])
	([3, 4, 9, 25, 31, 34], [9])
	([6, 21, 22, 23, 36, 44], [44])
	([2, 5, 11, 13, 42, 45], [42])
	([1, 3, 14, 35, 40, 42], [40, 42])


In [40]:
# 2 2 2 2 2
cnt = 1
for matched in matched_list:
    print(f'print predicted data [{cnt}]. {datetime.now()}')
    cnt1 = 1
    for m in matched:
        print_list(title=f"total_matched_{cnt}_model_{cnt1}", datas=m)
        cnt1 += 1
    cnt += 1

print predicted data [1]. 2025-05-21 09:50:01.488702
total_matched_1_model_1
	([1, 14, 19, 22, 35, 41], [])
	([19, 26, 29, 32, 38, 41], [])
	([15, 42, 38, 25, 31, 39], [42])
	([5, 12, 13, 33, 35, 39], [])
	([31, 34, 38, 43, 27, 7], [7])
total_matched_1_model_2
	([8, 23, 5, 36, 25, 34], [])
	([4, 10, 13, 23, 29, 40], [40])
	([9, 17, 22, 40, 42, 44], [9, 40, 42, 44])
	([27, 6, 9, 32, 45, 3], [9])
	([5, 29, 32, 41, 44, 45], [44])
total_matched_1_model_3
	([17, 19, 34, 35, 36, 40], [40])
	([13, 21, 32, 34, 41, 44], [44])
	([3, 4, 5, 20, 29, 42], [42])
	([24, 25, 11, 45, 44, 26], [24, 44])
	([8, 12, 16, 17, 22, 33], [])
total_matched_1_model_4
	([1, 2, 4, 6, 24, 37], [24])
	([33, 18, 9, 40, 39, 36], [9, 40])
	([30, 40, 15, 32, 24, 4], [24, 40])
	([6, 12, 14, 17, 43, 45], [])
	([19, 26, 44, 28, 38, 39], [44])
total_matched_1_model_5
	([1, 22, 23, 29, 36, 43], [])
	([12, 17, 18, 20, 29, 36], [])
	([3, 7, 8, 19, 31, 32], [7])
	([24, 23, 36, 31, 45, 33], [24])
	([37, 2, 13, 36, 4, 35], [])


In [38]:
# 2 1 1 2 2
cnt = 1
for matched in matched_list:
    print(f'print predicted data [{cnt}]. {datetime.now()}')
    cnt1 = 1
    for m in matched:
        print_list(title=f"total_matched_{cnt}_model_{cnt1}", datas=m)
        cnt1 += 1
    cnt += 1

print predicted data [1]. 2025-05-21 09:49:46.800735
total_matched_1_model_1
	([44, 40, 24, 1, 32, 17], [24, 40, 44])
	([2, 3, 4, 17, 31, 45], [])
	([3, 7, 20, 25, 37, 39], [7])
	([5, 8, 11, 23, 26, 29], [])
	([1, 12, 29, 31, 37, 44], [44])
total_matched_1_model_2
	([21, 37, 20, 40, 8, 44], [40, 44])
	([1, 13, 27, 28, 41, 45], [])
	([40, 30, 23, 26, 27, 10], [40])
	([9, 17, 21, 26, 38, 42], [9, 42])
	([6, 9, 11, 16, 17, 26], [9])
total_matched_1_model_3
	([6, 15, 18, 25, 33, 37], [])
	([10, 11, 25, 28, 38, 42], [42])
	([11, 12, 20, 21, 31, 40], [40])
	([2, 5, 10, 26, 43, 44], [44])
	([6, 12, 16, 23, 26, 37], [])
total_matched_1_model_4
	([8, 10, 11, 12, 35, 37], [])
	([2, 3, 15, 19, 23, 41], [])
	([38, 20, 28, 23, 33, 41], [])
	([10, 20, 32, 36, 37, 38], [])
	([18, 22, 26, 28, 32, 40], [40])
total_matched_1_model_5
	([9, 20, 31, 33, 35, 36], [9])
	([2, 5, 14, 38, 43, 44], [44])
	([9, 15, 22, 24, 29, 32], [9, 24])
	([1, 10, 21, 34, 39, 43], [])
	([19, 21, 18, 4, 27, 9], [9])


In [30]:
# 2 1 1 2 2
cnt = 1
for matched in matched_list:
    print(f'print predicted data [{cnt}]. {datetime.now()}')
    cnt1 = 1
    for m in matched:
        print_list(title=f"total_matched_{cnt}_model_{cnt1}", datas=m)
        cnt1 += 1
    cnt += 1

print predicted data [1]. 2025-05-21 09:48:27.283996
total_matched_1_model_1
	([19, 11, 43, 30, 16, 31], [])
	([22, 13, 35, 20, 21, 36], [])
	([12, 24, 29, 31, 33, 45], [24])
	([9, 14, 15, 19, 23, 38], [9])
	([1, 9, 20, 29, 39, 41], [9])
total_matched_1_model_2
	([10, 17, 18, 32, 35, 44], [44])
	([39, 6, 13, 25, 27, 44], [44])
	([7, 12, 14, 27, 33, 45], [7])
	([7, 26, 34, 35, 36, 42], [7, 42])
	([8, 18, 19, 28, 32, 38], [])
total_matched_1_model_3
	([7, 12, 16, 35, 39, 45], [7])
	([12, 10, 42, 44, 26, 40], [40, 42, 44])
	([5, 15, 24, 28, 32, 45], [24])
	([39, 8, 24, 10, 1, 23], [24])
	([7, 10, 16, 20, 32, 39], [7])
total_matched_1_model_4
	([6, 32, 42, 21, 45, 17], [42])
	([3, 37, 36, 23, 28, 12], [])
	([28, 29, 30, 35, 37, 44], [44])
	([27, 41, 33, 21, 32, 20], [])
	([5, 8, 15, 18, 21, 26], [])
total_matched_1_model_5
	([10, 13, 17, 29, 36, 45], [])
	([7, 16, 19, 24, 32, 36], [7, 24])
	([4, 20, 21, 28, 38, 44], [44])
	([3, 22, 32, 39, 44, 45], [44])
	([8, 20, 23, 39, 41, 14], [])


In [28]:
cnt = 1
for matched in matched_list:
    print(f'print predicted data [{cnt}]. {datetime.now()}')
    cnt1 = 1
    for m in matched:
        print_list(title=f"total_matched_{cnt}_model_{cnt1}", datas=m)
        cnt1 += 1
    cnt += 1

print predicted data [1]. 2025-05-21 09:48:06.649797
total_matched_1_model_1
	([2, 4, 15, 21, 25, 45], [])
	([32, 11, 2, 22, 25, 20], [])
	([5, 10, 14, 21, 25, 39], [])
	([7, 20, 21, 24, 42, 43], [7, 24, 42])
	([2, 8, 19, 32, 34, 38], [])
total_matched_1_model_2
	([9, 20, 24, 25, 32, 43], [9, 24])
	([44, 41, 6, 1, 2, 8], [44])
	([6, 23, 24, 26, 31, 41], [24])
	([9, 40, 35, 45, 39, 18], [9, 40])
	([3, 15, 20, 23, 24, 44], [24, 44])
total_matched_1_model_3
	([22, 36, 37, 39, 43, 45], [])
	([4, 6, 22, 25, 31, 40], [40])
	([1, 9, 25, 36, 43, 44], [9, 44])
	([1, 12, 23, 26, 37, 40], [40])
	([40, 9, 32, 15, 23, 26], [9, 40])
total_matched_1_model_4
	([11, 14, 20, 27, 37, 43], [])
	([8, 15, 29, 32, 34, 39], [])
	([21, 13, 9, 34, 38, 41], [9])
	([1, 38, 29, 13, 3, 32], [])
	([18, 19, 20, 26, 36, 39], [])
total_matched_1_model_5
	([6, 11, 25, 28, 14, 26], [])
	([11, 23, 30, 35, 39, 40], [40])
	([6, 8, 10, 15, 16, 43], [])
	([35, 25, 14, 4, 39, 19], [])
	([9, 13, 18, 33, 37, 42], [9, 42])


In [18]:
cnt = 1
for matched in matched_list:
    print(f'print predicted data [{cnt}]. {datetime.now()}')
    cnt1 = 1
    for m in matched:
        print_list(title=f"total_matched_{cnt}_model_{cnt1}", datas=m)
        cnt1 += 1
    cnt += 1

print predicted data [1]. 2025-05-21 09:44:17.619144
total_matched_1_model_1
	([16, 44, 27, 42, 14, 21], [42, 44])
	([10, 11, 20, 24, 33, 39], [24])
	([8, 11, 13, 19, 24, 31], [24])
	([2, 15, 16, 19, 22, 34], [])
	([25, 39, 23, 13, 4, 15], [])
total_matched_1_model_2
	([8, 39, 30, 14, 7, 3], [7])
	([7, 8, 24, 29, 37, 43], [7, 24])
	([2, 4, 14, 22, 35, 39], [])
	([5, 24, 28, 31, 38, 41], [24])
	([8, 26, 40, 45, 4, 20], [40])
total_matched_1_model_3
	([36, 30, 34, 43, 42, 17], [42])
	([10, 12, 19, 24, 37, 45], [24])
	([1, 4, 9, 23, 31, 39], [9])
	([2, 5, 8, 10, 19, 33], [])
	([18, 40, 17, 36, 38, 44], [40, 44])
total_matched_1_model_4
	([9, 35, 42, 2, 43, 38], [9, 42])
	([15, 27, 34, 36, 43, 45], [])
	([1, 2, 7, 24, 32, 41], [7, 24])
	([15, 21, 22, 29, 38, 44], [44])
	([8, 10, 13, 14, 35, 44], [44])
total_matched_1_model_5
	([1, 11, 24, 37, 41, 45], [24])
	([1, 7, 12, 14, 20, 39], [7])
	([6, 8, 19, 33, 36, 42], [42])
	([34, 24, 5, 40, 7, 10], [7, 24, 40])
	([10, 18, 19, 20, 35, 44], [44]

In [98]:
cnt = 1
for matched in matched_list:
    print(f'print predicted data [{cnt}]. {datetime.now()}')
    cnt1 = 1
    for m in matched:
        print_list(title=f"total_matched_{cnt}_model_{cnt1}", datas=m)
        cnt1 += 1
    cnt += 1

print predicted data [1]. 2025-05-21 09:20:10.895890
total_matched_1_model_1
	([3, 7, 9, 19, 27, 29], [7, 9])
	([17, 25, 33, 35, 37, 45], [])
	([15, 3, 33, 17, 8, 29], [])
	([8, 10, 14, 27, 38, 42], [42])
	([10, 11, 45, 1, 44, 12], [44])
total_matched_1_model_2
	([8, 12, 16, 26, 42, 44], [42, 44])
	([1, 10, 9, 27, 31, 13], [9])
	([4, 15, 17, 30, 39, 45], [])
	([20, 33, 14, 17, 39, 4], [])
	([11, 17, 19, 27, 36, 37], [])
total_matched_1_model_3
	([1, 6, 8, 16, 22, 43], [])
	([21, 27, 14, 9, 7, 40], [7, 9, 40])
	([30, 41, 37, 3, 8, 43], [])
	([1, 12, 17, 32, 42, 44], [42, 44])
	([27, 38, 22, 9, 23, 3], [9])
total_matched_1_model_4
	([29, 30, 44, 28, 18, 9], [9, 44])
	([21, 38, 29, 2, 9, 12], [9])
	([7, 17, 34, 38, 41, 42], [7, 42])
	([3, 14, 16, 19, 32, 45], [])
	([36, 8, 30, 40, 39, 28], [40])


In [61]:
cnt = 1
for matched in matched_list:
    print(f'print predicted data [{cnt}]. {datetime.now()}')
    cnt1 = 1
    for m in matched:
        print_list(title=f"total_matched_{cnt}_model_{cnt1}", datas=m)
        cnt1 += 1
    cnt += 1

print predicted data [1]. 2025-05-20 17:52:13.679866
total_matched_1_model_1
	([21, 33, 28, 15, 31, 26], [])
	([7, 9, 14, 16, 35, 38], [7, 9])
	([16, 44, 31, 40, 41, 10], [40, 44])
	([7, 21, 29, 33, 43, 45], [7])
	([13, 10, 8, 11, 6, 26], [])
total_matched_1_model_2
	([4, 29, 41, 21, 6, 40], [40])
	([8, 14, 23, 30, 31, 34], [])
	([6, 14, 17, 23, 28, 29], [])
	([9, 18, 22, 23, 33, 43], [9])
	([3, 11, 32, 36, 38, 39], [])
total_matched_1_model_3
	([7, 16, 26, 29, 30, 42], [7, 42])
	([12, 18, 22, 26, 35, 41], [])
	([7, 11, 18, 25, 42, 44], [7, 42, 44])
	([5, 21, 23, 31, 42, 45], [42])
	([42, 1, 4, 7, 11, 14], [7, 42])


In [49]:
cnt = 1
for matched in matched_list:
    print(f'print predicted data [{cnt}]. {datetime.now()}')
    cnt1 = 1
    for m in matched:
        print_list(title=f"total_matched_{cnt}_model_{cnt1}", datas=m)
        cnt1 += 1
    cnt += 1

print predicted data [1]. 2025-05-20 17:47:13.175427
total_matched_1_model_1
	([1, 2, 12, 28, 33, 38], [])
	([39, 1, 24, 43, 23, 26], [24])
	([2, 3, 4, 6, 31, 37], [])
	([2, 14, 22, 33, 34, 42], [42])
	([8, 12, 23, 25, 26, 40], [40])
total_matched_1_model_2
	([5, 9, 10, 27, 31, 40], [9, 40])
	([1, 12, 14, 16, 24, 44], [24, 44])
	([2, 22, 28, 30, 34, 43], [])
	([9, 10, 12, 25, 33, 35], [9])
	([43, 17, 7, 33, 25, 2], [7])
total_matched_1_model_3
	([7, 14, 16, 23, 26, 42], [7, 42])
	([21, 23, 25, 26, 31, 35], [])
	([1, 15, 31, 33, 35, 45], [])
	([17, 26, 15, 40, 30, 44], [40, 44])
	([7, 9, 24, 29, 35, 40], [7, 9, 24, 40])


In [240]:
[
([3, 5, 10, 11, 13, 26], [3, 11]),
([36, 10, 7, 32, 1, 11], [7, 11]),
([20, 41, 21, 14, 6, 40], [6]),
([12, 15, 19, 21, 34, 36], [12]),
([27, 40, 35, 5, 45, 37], [])]


[([3, 9, 11, 28, 29, 30], [3, 11]),
([1, 6, 12, 22, 28, 32], [6, 12]),
([1, 11, 25, 28, 32, 45], [11]),
([7, 22, 27, 28, 38, 39], [7]),
([39, 12, 24, 11, 31, 25], [11, 12])]


[([3, 4, 8, 21, 26, 27], [3]),
([3, 4, 14, 18, 28, 37], [3]),
([3, 15, 17, 23, 33, 36], [3, 17]),
([44, 30, 5, 41, 10, 3], [3]),
([1, 6, 13, 17, 30, 45], [6, 17])
]

[([3, 4, 8, 21, 26, 27], [3]),
 ([3, 4, 14, 18, 28, 37], [3]),
 ([3, 15, 17, 23, 33, 36], [3, 17]),
 ([44, 30, 5, 41, 10, 3], [3]),
 ([1, 6, 13, 17, 30, 45], [6, 17])]

In [ ]:
# all : save model
for model in models:
    if model is not None:
        model[0].save()

In [ ]:
# all : model info
#################
# model info
#################
for model in models:
    if model is not None:
        print(f'model:\n\t{model[0].model.summary()}')
        print(f'layer:\n\t{model[1]}')

In [55]:
# all : model test
mode = "back-test"
if mode == 'back-test':
    for model in models:
        if model is not None:
            greed_prediction_number_set = model[0].predict_numbers("greed", trial=1)
            random_pred_set = model[0].predict_randomely(trial=1)
            model[0].evaluate(greed_prediction_number_set)
            print("---------Random baseline-------------")
            model[0].evaluate(random_pred_set)

In [ ]:
for i in range(3):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
for i in range(5):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
# one : create model and training
id = 6
print(f'start to train. [id={id}]')
models[6] = create_model_v1(id=6, dataset=dataset, epoch=10, verbose=True)
print(f'model = {models[6]}')
print(f'end to train. [id={id}]')

In [22]:
# [3, 6, 7, 11, 12, 17]

In [29]:
new1 = flat_data(prediction_number_set22)
new_dict = {}
for n in new1:
    if n in new_dict:
        new_dict[n] += 1
    else:
        new_dict[n] = 1
# print(f'new_dict = {new_dict}')
top, all = get_sorted_n_values(new_dict)
print(f'all = {all}')
print(f'top = {top}')
# print(f'new_dict = {new_dict}')
bottom, all = get_sorted_n_values(new_dict, is_sort=False)
print(f'bottom = {bottom}')

NameError: name 'prediction_number_set22' is not defined

In [ ]:
for i in range(5):
    print(f'{i} : {get_random_in_list(prediction_number_set2, 5)}')

In [ ]:
print_data(title='greed_predicted',
           data_set=greed_prediction_number_set,
           add_val=0, need_sort=True)
print(f'greed_prediction_number_set\n\t\t{greed_prediction_number_set}')

In [ ]:
# when 
"""
trial=20 # help = "how much trials to generate")
training_length=1 # default = 0.9)
epoch=100 # default = 3
batch=3 # default = 1
model='lstm2' # help = "lstm or lstm2")
hid_dim = 128
"""
for i in range(6):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
print(f'model={model}')

In [ ]:
random_pred_set = model2.predict_randomely(trial=1)
print(f'prediction_number_set = {random_pred_set}')

In [ ]:
# 1170 : 3·13·28·34·38·42

last = [
    [6, 8, 37, 40, 41, 44],
    [8, 10, 20, 25, 33, 37],
    [4, 18, 27, 32, 40, 43],
    [2, 8, 10, 31, 33, 35],
    [15, 20, 28, 38, 41, 45],
    [12, 26, 13, 5, 31, 38]
]

In [ ]:
#1171 : [3, 6, 7, 11, 12, 17]